In [ ]:
import networkx as nx
import random


# SI模型的传播参数
def SI_model(G, initial_infected, infection_probability, max_steps=20):

    infected_count = [len(initial_infected)]  # 用来记录每个时间步感染节点的数量
    infected_nodes = set(initial_infected)  # 初始感染者
    susceptible_nodes = set(G.nodes) - infected_nodes  # 易感节点
    
    # 模拟传播过程
    for t in range(max_steps):
        newly_infected = set()
        
        # 遍历每个感染节点，并尝试传染其未感染的邻居
        for node in infected_nodes:
            uninfected_nodes = susceptible_nodes & set(G.neighbors(node))  # 获取邻居节点中的未感染节点
            for uninfected in uninfected_nodes:
                if random.random() < infection_probability:
                    newly_infected.add(uninfected)
            susceptible_nodes -= newly_infected  # 更新易感节点
            
        # 更新感染节点集
        infected_nodes.update(newly_infected)
        num_infected = len(G.nodes()) - len(susceptible_nodes)
        infected_count.append(num_infected)

    return infected_count

In [ ]:
import networkx as nx


# 定义SIR模型
def SIR_model(G, beta, gamma, initial_infected, max_iter=100):
    status = {node: 0 for node in G.nodes()}  # 节点状态初始化: 所有节点初始为易感者 (S)
    for node in initial_infected:
        status[node] = 1  # 初始感染节点设为感染者 (I)
    infected_nodes = set(initial_infected)  # 初始感染者
    recoverd_ndes = set()
    susceptible_nodes = set(G.nodes) - infected_nodes  # 易感节点
    for _ in range(max_iter):
        new_status = status.copy()
        newly_infected = set()
        newly_recovered = set()
        for node in infected_nodes:
            uninfected_nodes = susceptible_nodes & set(G.neighbors(node))  # 获取邻居节点中的未感染节点
            for uninfected in uninfected_nodes:
                if np.random.rand() < beta:  # 根据传播率感染邻居
                    new_status[uninfected] = 1
                    newly_infected.add(uninfected)
            if np.random.rand() < gamma:  # 根据康复率康复
                new_status[node] = 2
                newly_recovered.add(node)
                if node in newly_infected:
                    newly_infected.remove(node)         
            susceptible_nodes = susceptible_nodes - newly_infected
            susceptible_nodes.update(newly_recovered)
        infected_nodes.update(newly_infected)   # 更新被感染节点的数量
        status = new_status

    return status


# 计算每个节点的重要性
def calculate_node_importance(G, beta, gamma, max_iter=100, num_simulations=10):
    node_importance = {node: 0 for node in G.nodes()}  # 初始化每个节点的影响力为0
    
    for node in G.nodes():
        total_infected = 0
        for _ in range(num_simulations):  # 对每个节点进行多次模拟，求平均感染数
            status_node = SIR_model(G, beta, gamma, [node], max_iter)
            infected_count = len([status for status in status_node.values() if status == 1])  # 1表示被感染
            total_infected += infected_count
        node_importance[node] = total_infected / num_simulations  # 计算平均影响力
    
    return node_importance  # 存储每个节点可感染节点的数量


# 排序并返回节点的重要性排名
def get_node_ranking(node_importance):
    
    return dict(sorted(node_importance.items(), key=lambda iterm: iterm[1], reverse=True))

In [ ]:
from collections import Counter


def monotony (centrality, numbers_nodes):
    n = numbers_nodes
    ns = 0
    # 同一个中心性值所出现的次数
    counter = Counter(centrality.values())
    # 过滤出出现次数 >= 1 的元素（实际来说，所有元素都会被包括）
    filtered_dict = {key: value for key, value in counter.items() if value > 1}
    # 获取相同值所出现的次数
    numbers_same_value = [num for num in filtered_dict.values()]
    for data in numbers_same_value:
        ns += data * (data - 1)
    monotony = (1 - ns / (n * (n - 1))) ** 2
    return (monotony)

In [ ]:
import networkx as nx


def valid_degree(G):
    node_degree = dict(G.degree())
    sorted_node = dict(sorted(node_degree.items(), key=lambda item: item[1], reverse=True))
    sequence_node = list(sorted_node.keys())
    emerged_nodes = set()
    valid_degree = {node : 0 for node in G.nodes()}
    valid_degree[sequence_node[0]] = node_degree[sequence_node[0]]
    for node in sequence_node:
        # 当网络中所有节点都已经出现过则令之后节点的有效度值全为0
        if len(emerged_nodes) == len(G.nodes()):
            valid_degree[node] = 0
            continue
        # 计算当前节点
        neighbor_node = set()
        for neighbor in G.neighbors(node):
            neighbor_node.add(neighbor)
        # 的有效度值 = 邻居节点中未出现过的节点
        valid_degree[node] = len(neighbor_node - emerged_nodes)
        emerged_nodes.update(neighbor_node)
        
    return valid_degree
    

In [ ]:
import networkx as nx


def valid_degree(G):
    node_degree = dict(G.degree())

    # 获取具有最大度值的节点
    max_node = max(node_degree, key=node_degree.get)

    # 将度数最大的节点作为初始化出现节点
    emerged_nodes = set(G.neighbors(max_node))
    valid_degree = {node : 0 for node in G.nodes()}
    valid_degree[max_node] = node_degree[max_node]

    
    transacted_nodes = {max_node}
    untransacted_nodes = set(G.nodes()) - transacted_nodes

    while(untransacted_nodes):
        current_valid = {}
        for node in untransacted_nodes:
                
            # 计算当前节点的有效性
            current_valid[node] = len(set(G.neighbors(node)) - emerged_nodes)
            
        # 获取未处理节点中具有最大有效值的节点
        max_node = max(current_valid, key=current_valid.get) 
        valid_degree[max_node] = current_valid[max_node]
        
        # 将最大有效性节点的邻居节点更新到已出现的节点里
        emerged_nodes.update(set(G.neighbors(max_node)))
        
        # 更新已经处理过的和未处理过的节点
        transacted_nodes.add(max_node)
        untransacted_nodes.remove(max_node)
        
        # 当网络中所有节点都已经出现过则令之后节点的有效度值全为0
        if len(emerged_nodes) == len(G.nodes()):
            for node in untransacted_nodes:
                valid_degree[node] = 0
            return valid_degree

    return valid_degree
    